In [5]:
import numpy as np
from pyproj import Geod
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import plotly.express as px
from shapely.geometry import LineString, Point, Polygon

degree_sign = u"\N{DEGREE SIGN}"
def rad2dms(rad):
    dd = np.rad2deg(rad)
    dd = dd
    deg = int(np.trunc(dd))
    mnt = int(np.trunc((dd-deg) * 60))
    sec = ((dd-deg) * 60 - mnt) * 60
    mnt = abs(mnt)
    sec = abs(sec)
    if sec > 59.99999:
        sec = 0
        mnt += 1
    if sec < 10:
        sec = f"0{sec:.5f}"
    else:
        sec = f"{sec:.5f}"
    if mnt < 10:
        mnt = f"0{mnt}"
    dms = (f"{deg}{degree_sign} {mnt}' {sec}''")
    return dms

def deg2dms(dd):
    deg = int(np.trunc(dd))
    mnt = int(np.trunc((dd-deg) * 60))
    sec = ((dd-deg) * 60 - mnt) * 60
    mnt = abs(mnt)
    sec = abs(sec)
    if sec < 10:
        sec = f"0{sec:.5f}"
    else:
        sec = f"{sec:.5f}"
    if mnt < 10:
        mnt = f"0{mnt}"
    dms = (f"{deg}{degree_sign} {mnt}' {sec}''")
    return dms

a = 6378137.0 
f = 1/298.257223563
e2 = f * (2 - f)

phi_1_deg = 53 + 45/60
lam_1_deg = 15 + 15/60
s = [40000, 100000, 40000, 100000]
Az_1_deg = [0, 90, 180, 270]

phi_1 = np.deg2rad(phi_1_deg)
lam_1 = np.deg2rad(lam_1_deg)
Az_1 = np.deg2rad(Az_1_deg)

def M_and_N(phi):
    sin_phi = np.sin(phi)
    M = a * (1 - e2) / (1 - e2 * sin_phi**2)**(3/2)
    N = a / np.sqrt(1 - e2 * sin_phi**2)
    return M, N

def kivioj(phi_1, lam_1, Az_1, s, n=1000):
    ds = s / n

    phi = phi_1
    lam = lam_1
    Az = Az_1

    for i in range(n):
        M, N = M_and_N(phi)
        dphi = ds * np.cos(Az) / M
        dAz = ds * np.sin(Az) * np.tan(phi) / N

        phi_mid = phi + dphi / 2
        Az_mid = Az + dAz / 2

        M_mid, N_mid = M_and_N(phi_mid)
        dphi_mid = ds * np.cos(Az_mid) / M_mid
        dlam_mid = ds * np.sin(Az_mid) / (N_mid * np.cos(phi_mid))
        dAz_mid = ds * np.sin(Az_mid) * np.tan(phi_mid) / N_mid

        phi += dphi_mid
        lam += dlam_mid
        Az += dAz_mid

    return phi, lam, Az

g = Geod(ellps='WGS84')
nr = ['1', '2', '3', '4', '1*']

phis_kiv = [phi_1_deg]
lambdas_kiv = [lam_1_deg]
azimuths_kiv = [Az_1_deg[0]]

phis_kiv_dms = [deg2dms(phi_1_deg)]
lambdas_kiv_dms = [deg2dms(lam_1_deg)]
azimuths_kiv_dms = [deg2dms(Az_1_deg[0])]

for i in range(4):
    phi, lam, Az = kivioj(phi_1, lam_1, Az_1[i], s[i])

    phis_kiv.append(np.rad2deg(phi))
    lambdas_kiv.append(np.rad2deg(lam))
    azimuths_kiv.append(np.rad2deg(Az))

    phis_kiv_dms.append(rad2dms(phi))
    lambdas_kiv_dms.append(rad2dms(lam))
    azimuths_kiv_dms.append(rad2dms(Az))

    phi_1 = phi
    lam_1 = lam

'''
phi_1 = np.deg2rad(phi_1_deg)
lam_1 = np.deg2rad(lam_1_deg)

phis_fwd = [phi_1_deg]
lambadas_fwd = [lam_1_deg]
azimuths_fwd = [Az_1_deg[0]]

phis_fwd_dms = [deg2dms(phi_1_deg)]
lambadas_fwd_dms = [deg2dms(lam_1_deg)]
azimuths_fwd_dms = [deg2dms(Az_1_deg[0])]

for i in range(4):
    phi, lam, Az = g.fwd(lam_1, phi_1, Az_1[i], s[i])

    phis_fwd.append(np.rad2deg(phi))
    lambadas_fwd.append(np.rad2deg(lam))
    azimuths_fwd.append(np.rad2deg(Az))
    
    phis_fwd_dms.append(rad2dms(phi))
    lambadas_fwd_dms.append(rad2dms(lam))
    azimuths_fwd_dms.append(rad2dms(Az))

fig = go.Figure(go.Table(header = dict(values = ['nr', 'phi', 'lambda', 'azymut']), cells = dict(values = [nr, phis_fwd_dms, lambadas_fwd_dms, azimuths_fwd_dms])))
fig.update_layout(title = 'Współrzędne punktów obliczone funkcją fwd')
fig.show()
'''

fig = go.Figure(go.Table(
    header = dict(values = ['nr', 'phi', 'lambda', 'azymut']), 
    cells = dict(values = [nr, phis_kiv_dms, lambdas_kiv_dms, azimuths_kiv])
))
fig.update_layout(
    title = 'Współrzędne punktów obliczone algorytmem Kivioja', 
    width = 1000, height = 400
)
fig.show()

fig = go.Figure(
    go.Scattermapbox(
        lat = phis_kiv,
        lon = lambdas_kiv,
        mode = 'markers+lines',
        marker = dict(size = 10, color = 'red'),
        line = dict(width = 2, color = 'red'),
        text = nr,
        hoverinfo='text'))
fig.update_layout(
    title = 'Scattermapbox - Algorytm Kivioja',
    mapbox_style = "open-street-map",
    width = 2000, 
    height = 1000,
    mapbox = dict(
        center = go.layout.mapbox.Center(
            lat = phi_1_deg,
            lon = lam_1_deg
        ),
        zoom = 7
    ),
)
fig.show()

print(phis_kiv[-1], lambdas_kiv[-1])

phi_dif = phis_kiv[-1] - phi_1_deg
lam_dif = lambdas_kiv[-1] - lam_1_deg
total_length = g.line_length([lam_1_deg, lambdas_kiv[-1]], [phi_1_deg, phis_kiv[-1]])
phi_dif = deg2dms(phi_dif)
lam_dif = deg2dms(lam_dif)

# print(f"Różnica współrzędnych: [{phi_dif}, {lam_dif}], długość linii: {total_length:.5f} m")


def vincenty(BA,LA,BB,LB):
    b = a * np.sqrt(1-e2)
    f = 1-b/a
    dL = LB - LA
    UA = np.arctan((1-f)*np.tan(BA))
    UB = np.arctan((1-f)*np.tan(BB))
    L = dL
    while True:
        sin_sig = np.sqrt((np.cos(UB)*np.sin(L))**2 +\
            (np.cos(UA)*np.sin(UB) - np.sin(UA)*np.cos(UB)*np.cos(L))**2)
        cos_sig = np.sin(UA)*np.sin(UB) + np.cos(UA) * np.cos(UB) * np.cos(L)
        sig = np.arctan2(sin_sig,cos_sig)
        sin_al = (np.cos(UA)*np.cos(UB)*np.sin(L))/sin_sig
        cos2_al = 1 - sin_al**2
        cos2_sigm = cos_sig - (2 * np.sin(UA) * np.sin(UB))/cos2_al
        C = (f/16) * cos2_al * (4 + f*(4 - 3 * cos2_al))
        Lst = L
        L = dL + (1-C)*f*sin_al*(sig+C*sin_sig*(cos2_sigm+
            C*cos_sig*(-1 + 2*cos2_sigm**2)))
        if abs(L-Lst)<(0.000001/206265):
            break
    
    u2 = (a**2 - b**2)/(b**2) * cos2_al
    A = 1 + (u2/16384) * (4096 + u2*(-768 + u2 * (320 - 175 * u2)))
    B = u2/1024 * (256 + u2 * (-128 + u2 * (74 - 47 * u2)))
    d_sig = B*sin_sig * (cos2_sigm + 1/4*B*(cos_sig*(-1+2*cos2_sigm**2)\
            - 1/6 *B*cos2_sigm * (-3 + 4*sin_sig**2)*(-3+4*cos2_sigm**2)))
    sAB = b*A*(sig-d_sig)
    A_AB = np.arctan2((np.cos(UB) * np.sin(L)),(np.cos(UA)*np.sin(UB) - np.sin(UA)*np.cos(UB)*np.cos(L)))
    A_BA = np.arctan2((np.cos(UA) * np.sin(L)),(-np.sin(UA)*np.cos(UB) + np.cos(UA)*np.sin(UB)*np.cos(L))) + np.pi
    return sAB, A_AB, A_BA

lengths_vin = []
az_vin = []
az_odw_vin = []

for i in range(4):
    if i == 3:
        j = 0
    else:
        j = i + 1
    
    length_ij, az_ij, az_rev_ij = vincenty(np.deg2rad(phis_kiv[i]), np.deg2rad(lambdas_kiv[i]), np.deg2rad(phis_kiv[j]), np.deg2rad(lambdas_kiv[j]))
    lengths_vin.append(f"{length_ij:.5f}")
    if az_ij < 0:
        az_ij += 2 * np.pi
    if az_rev_ij < 0:
        az_rev_ij += 2 * np.pi
    az_vin.append(rad2dms(az_ij))
    az_odw_vin.append(rad2dms(az_rev_ij))

    print(f"Odcinek {i+1} - {j+1}: {length_ij:.3f} m, azymut: {np.rad2deg(az_ij)}, azymut odwrotny: {np.rad2deg(az_rev_ij)}")

length41, az41, az_odw41 = vincenty(np.deg2rad(phis_kiv[3]), np.deg2rad(lambdas_kiv[3]), np.deg2rad(phis_kiv[0]), np.deg2rad(lambdas_kiv[0]))
length = [f"{length41:.5f}"]
az_deg = rad2dms(az41)
az_odw_deg = rad2dms(az_odw41)

# Tabela odległości i azymutów 4 - 1
fig = go.Figure(go.Table(
header = dict(values = ['A - B', 'Odl. AB [m]', 'Az. AB', 'Az. odw. AB']), 
cells = dict(values = ['4 - 1', length, az_deg, az_odw_deg])
))
fig.update_layout(
title = 'Algorytm Vincentego', 
width = 1000, height = 300
)
fig.show()

phis_vin = phis_kiv[0:4]
lambdas_vin = lambdas_kiv[0:4]
azimuths_vin = azimuths_kiv[0:4]

phis_vin_dms = phis_kiv_dms[0:4]
lambdas_vin_dms = lambdas_kiv_dms[0:4]
azimuths_vin_dms = azimuths_kiv_dms[0:4]

phi_vin5, lam_vin5, Az_vin5 = kivioj(np.deg2rad(phis_kiv[3]), np.deg2rad(lambdas_kiv[3]), az41, length41)
if Az_vin5 < 0:
    Az_vin5 += 2 * np.pi
phis_vin.append(np.rad2deg(phi_vin5))
lambdas_vin.append(np.rad2deg(lam_vin5))
azimuths_vin.append(rad2dms(Az_vin5))

phis_vin_dms.append(rad2dms(phi_vin5))
lambdas_vin_dms.append(rad2dms(lam_vin5))
azimuths_vin_dms.append(rad2dms(Az_vin5))

# Tabela skorygowanych współrzędnych
fig = go.Figure(go.Table(
header = dict(values = ['nr', 'phi', 'lambda', 'azymut']), 
cells = dict(values = [nr, phis_vin_dms, lambdas_vin_dms, azimuths_vin])
))
fig.update_layout(
title = 'Współrzędne właściwe punktów obliczone algorytmem Vincentego', 
width = 1000, height = 400
)
fig.show()


fig = go.Figure(
    go.Scattermapbox(
        lat = phis_vin,
        lon = lambdas_vin,
        mode = 'markers+lines',
        marker = dict(size = 10, color = 'blue'),
        line = dict(width = 2, color = 'blue'),
        text = nr,
        hoverinfo='text'))
fig.update_layout(
    title = 'Scattermapbox - Algorytm Vincentego',
    mapbox_style = "open-street-map",
    width = 2000, 
    height = 1000,
    mapbox = dict(
        center = go.layout.mapbox.Center(
            lat = phi_1_deg,
            lon = lam_1_deg
        ),
        zoom = 7
    ),
)
fig.show()

geod = Geod('+a=6378137 +f=0.0033528106647475126')

area, perimeter = g.geometry_area_perimeter(
    Polygon(
        LineString([
            Point(lambdas_vin[i], phis_vin[i]) for i in range(len(lambdas_vin))])
        )
    )
area = abs(area)
area_km = area / 1000000
perimeter_km = perimeter / 1000
area = f"{area:.3f}"
area_km = f"{area_km:.3f}"
perimeter = f"{perimeter:.3f}"
perimeter_km = f"{perimeter_km:.3f}"

fig = go.Figure(go.Table(header = dict(values = [r'','Pole', r'Obwód']), cells = dict(values = [['[km]', '[m]'], [area, area_km], [perimeter, perimeter_km]])))
fig.update_layout(title = 'Pole i obwód wielokąta', width = 1000, height = 300)
fig.show()



53.73070903207916 15.263419951079845
Odcinek 1 - 2: 40000.000 m, azymut: 0.0, azymut odwrotny: 180.0
Odcinek 2 - 3: 100000.000 m, azymut: 90.00000000009787, azymut odwrotny: 271.2384278825927
Odcinek 3 - 4: 40000.000 m, azymut: 180.0, azymut odwrotny: 360.0
Odcinek 4 - 1: 100862.551 m, azymut: 271.23030946924604, azymut odwrotny: 89.99752749815782
